In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding,EarlyStoppingCallback
from peft import get_peft_model,get_peft_config, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import Dataset, DatasetDict
import pandas as pd
import torch
import evaluate
import numpy as np
from trl import SFTTrainer
import mlflow

In [2]:
TOKEN = "{Enter token here}"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-instruct", padding_side="right", token=TOKEN,)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
    llm_int8_skip_modules=None
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-instruct",
    quantization_config=bnb_config,
    token=TOKEN,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.config.pad_token_id = tokenizer.pad_token_id
model.gradient_checkpointing_enable()

In [7]:
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']
# PEFT Configuration
peft_config = LoraConfig(
    r=10,
    target_modules = target_modules,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 16,509,440 || all params: 3,229,259,264 || trainable%: 0.5112
None


/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:500: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [8]:
# Load and prepare datasets
df1 = pd.read_csv("dataset/dataset-checkpoint-1-processed.csv")
df2 = pd.read_csv("dataset/dataset-checkpoint-2-processed.csv")

In [9]:
# Merge and randomize set

rd_df = pd.concat([df1, df2])
rd_df_sample = rd_df.sample(frac=1, random_state=42) 

rd_df_sample

,Id,comment,quality,sentiment,positive-attribute,negative-attribute,processed_comments
544,103621.0,"Avoid this guy, he's a fool. All these people ...",awful,0,NaN,unhelpful,"Avoid this guy, he's a fool. All these people ..."
410,277557.0,This is the best class at UVSC. Don't listen t...,awesome,1,clear,NaN,This is the best class at UVSC. Don't listen t...
351,NaN,Professor Lee breaks down complex topics into ...,NaN,1,clear,NaN,Professor Elliot breaks down complex topics in...
1281,304408.0,DO NOT TAKE!!! She is a very knowledgeable pro...,average,1,knowledgeable,NaN,DO NOT TAKE!!! She is a very knowledgeable pro...
598,1307460.0,Dr. Parker seems as though he wants you to lea...,poor,0,NaN,unfair,Dr. Harper seems as though he wants you to lea...
...,...,...,...,...,...,...,...
1168,NaN,This professor truly cares about the subject m...,NaN,1,passionate,NaN,This professor truly cares about the subject m...
466,103621.0,This guy is a terrible professor. He is consta...,awful,0,NaN,confusing,This guy is a terrible professor. He is consta...
816,NaN,Professor Smith really loves what they teach! ...,NaN,1,passionate,NaN,Professor Sidney really loves what they teach!...
1496,NaN,This professor seemed completely disinterested...,NaN,0,NaN,unhelpful,This professor seemed completely disinterested...


In [10]:
rd_df_sample['instruction'] = 'Provide a sentiment score for the following comment. The score should be either 0 or 1 (For negative or positive). Output just the score and nothing more. Comment: '+ rd_df_sample['processed_comments']

print(rd_df_sample['instruction'].iloc[1])

Provide a sentiment score for the following comment. The score should be either 0 or 1 (For negative or positive). Output just the score and nothing more. Comment: This is the best class at UVSC. Don't listen to anyone that tells you it's hard -- as long as you can memorize, you'll be fine. There's a ton of material, but nothing is difficult to understand. Emerson & Homan are AWESOME. You DO have to live at school to get an A though, so be ready for 12hr study-sessions!


In [11]:
template = """

### Instruction:

{}

### Response:\n"""

In [12]:
# Filter out rows where 'sentiment' is NaN
rd_df_sample = rd_df_sample[rd_df_sample['sentiment'].notna()]

# Create the prompt format using special tokens
rd_df_sample['prompt'] = rd_df_sample["instruction"].apply(
    lambda x: f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
              f"You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
              f"{x}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
)

# Rename 'sentiment' to 'response'
rd_df_sample.rename(columns={'sentiment': 'response'}, inplace=True)
rd_df_sample['response'] = rd_df_sample['response'].astype(str) + "\n<|eot_id|>"

# Select only the 'prompt' and 'response' columns
rd_df_sample = rd_df_sample[['prompt', 'response']]

In [13]:
print(rd_df_sample['prompt'].iloc[2])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
Provide a sentiment score for the following comment. The score should be either 0 or 1 (For negative or positive). Output just the score and nothing more. Comment: Professor Elliot breaks down complex topics into manageable pieces, making it easy to understand even challenging concepts. He provides lots of real-world examples which helped me grasp the material. The way he explains things just makes sense!<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [14]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(rd_df_sample, test_size=0.07, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict for easy handling of both sets
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [15]:
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [16]:
import os
temp_output_dir = "/tmp/training_checkpoints"
os.makedirs(temp_output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=temp_output_dir,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=5,
    save_steps=15000000,
    logging_steps=5,
    num_train_epochs = 3.0,
    load_best_model_at_end=True,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [17]:
# Set up the early stopping callback with patience
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations to wait for improvement
    early_stopping_threshold=0.0  # Minimum change to qualify as an improvement
)

In [18]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field="prompt",
    max_seq_length=150,
    args=training_args,
    callbacks=[early_stopping_callback],
)

/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the o

Map:   0%|          | 0/3976 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [19]:
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.float32)

In [20]:
with mlflow.start_run(run_name='run_name_of_choice'):
    trainer.train()

/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*a

Step,Training Loss,Validation Loss
5,4.322300,4.295690
10,4.385400,4.291339
15,4.154400,4.233607
20,4.025600,4.158978
25,3.958800,4.067419
30,3.868000,3.957089
35,3.783300,3.851639
40,3.746900,3.776284
45,3.859500,3.667710
50,4.142700,3.567793


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [21]:
from peft import LoraModel

lora_model = LoraModel(model,peft_config,"sentiment-llama3-full")

# Define the directory and naming for saving
save_directory = "loras/sentiment-llama3-full"

# Save only the LoRA weights
lora_model.save_pretrained(save_directory)

print(f"LoRA weights saved to {save_directory}")

/home/jasonspc/course-assn/cs3450/cs3450-team-1-project/ai-backend/train/train-env/lib/python3.12/site-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


LoRA weights saved to loras/sentiment-llama3-full


In [22]:
import shutil
shutil.rmtree(temp_output_dir)